# ELE6310 - Assignment 2 - Data-Flow and Design Space

#### Name: 
#### Student ID: 

In [ ]:
#@title Mount your Google Drive
%matplotlib inline
from importlib import reload


from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/ELE6310/A2" #@param {type:"string"}
!ln -Ts $folder /content/A2 2> /dev/null

# Add the assignment folder to Python path
if '/content/A2' not in sys.path:
    sys.path.insert(0, '/content/A2')

# Install requirements
!uv pip install -qr /content/A2/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

## 1- Knowledge Distillation

In [ ]:
import solution
import torch
import torch.nn as nn 
from transformers import AutoModel
import timm
import numpy as np
import random
from matplotlib import pyplot as plt
import os

In [ ]:
from common.utils import evaluate, model_size, load_CIFAR10_distill_dataset, DinoCifar, model_to_quant
from common.resnet import resnet_from_scratch

In [ ]:
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

* Load the teacher DINOv2 model

In [ ]:
#Download model weights from https://drive.google.com/file/d/15e3zlHZbjeNcoBbprmZ1Zt4F-KPqbgEa/view?usp=sharing
model = AutoModel.from_pretrained('facebook/dinov2-small')
num_classes = 10
classifier = nn.Linear(384, num_classes)
teacher_model = DinoCifar(model, classifier)
device = torch.device('cuda:0')
teacher_model.to(device)
teacher_model.load_state_dict(torch.load("dino_small_cifar10.pth", map_location="cuda"))


In [ ]:
data_config = timm.data.resolve_model_data_config(model)
train_transform = timm.data.create_transform(
    **data_config,
    is_training=True
)
test_transform = timm.data.create_transform(
    **data_config,
    is_training=False
)

train_loader, test_loader, calibration_loader = load_CIFAR10_distill_dataset(
    batch_size=256,
    teacher_train_transform=train_transform, 
    teacher_test_transform=test_transform,
    student_train_transform=None,
    student_test_transform=None,
)

* Load the student ResNet model (customizable)

In [ ]:
student_model = resnet_from_scratch(layers=[6, 6, 6], channels=[32, 64, 128], num_classes=10)
student_model.to(device)

accuracy = evaluate(student_model, test_loader, device)
print("test accuracy of fp model:", accuracy, "model size:", model_size(student_model))


* Quantization of the student model

In [ ]:
method='sym'
act_N_bits=8
weight_N_bits=8

quantized_student_model = model_to_quant(student_model, calibration_loader, act_N_bits, weight_N_bits,method, device)

* Distillation process

In [ ]:
quantized_student_model, hist = solution.distill_from_frozen_teacher(
    teacher_model, quantized_student_model, train_loader, test_loader,
    epochs=10, lr=0.1, temperature=2.0, alpha_soft=0.2
)

* Use a method of your choice to find the optimal energy consumption for ResNet with a constraint on test accuracy above 85%.

Any reasonable attempt at exploring the design space will give you full marks. Better approaches/results will be considered for bonus points.

In [ ]:
accuracy = evaluate(quantized_student_model, test_loader, device)
print("test accuracy after training:", accuracy, "model size:", model_size(quantized_student_model))


After fine-tuning, save the model and generate the YAML files for each layers of the student model. Then you can use `run_Accelergy` to estimate the energy consumption of student network.

In [ ]:
solution.generate_resnet_layers(quantized_student_model, base_path='common/layer_prob_base.yaml',  path='Q3/prob')